In order to decide the best real estate to buy for rental we should figure out what will give us the most Utility over time.  This Utility calculation will factor in Fixed Costs of the property, Fixed Income of the property, Property tax + Appreciation, and Happiness the property brings.  Below we'll walk through each of these aspects and combine them to get a metric for Utility relative to key parameters such as estimated rental rate, property tax, initial cost of the property, appreciation rate estimate, etc.

At the end, after we have calculated these values, we then look at gross_monetary_end_amount at the end of a certain time frame as the Income - Costs, assumed to grow at the MARKET_RATE (an adjustable parameter indicating the expected appreciation of this capital if it were invested in the market, e.g. estimated time value of money), plus the end appreciated value of the property.  This gross_monetary_end_amount metric would give us the total expected cash in hand if we ran the property for a given number of years (NUMBER_YEARS_RENTED), then sold it and gathered all of the money we had made from the rental income minus costs in the market.

It should be noted that none of this accounts for distributional risk of investment, we choose rates that are typically "safe" over the time span that we selected.  For example we would never expect 5% annual return to be a safe average market rate for a 1 year time span but over a 15 year time span it's generally a safe expectation.

Before we get into the exact calculations, we will define the following function that will calculate our time value of money for a recurring cost. This is essentially answering the question "How much money would I have at the end of **duration** if I invested a recurring **amount** of money in an asset that appreciated at a certain **market_appreciation_rate**. Optionally, you can also enter a **percent_increase_in_amount_over_time** which is for if you expect the amount that you invest each year to increase (e.g. if your salary is expected to increase each year).

For example if you put your yearly income 100000, into the S&P 500 each year and assumed 7% growth per year on average, for 10 years, and you expected your yearly income to increase by 2% per year, the below function would give you the amount of money you would have at the end of that time period if you called it with **compute_time_value_of_recurring_cost(100000, 0.02, 0.07, 10)**.  We will plug in our costs and our incomes to this to get the amount they are worth at final liquidation.  Basically this helps us account for the fact that paying money later is better.

In [1]:
import numpy as np
def compute_time_value_of_recurring_cost(amount, 
                                         percent_increase_in_amount_over_time, 
                                         market_appreciation_rate, 
                                         duration):
    remainder = amount*(duration - int(duration))
    
    if duration != int(duration):
        #if we have something like 10.5 years and need to amortize the interest
        per_year_amount_vect = np.array([amount]*int(duration) + [remainder])
    else:
        #if we don't have a clean integer number of years/months/increments
        per_year_amount_vect = np.array([amount]*int(duration))
    percent_increase_in_amount_over_time
    exponential_factors = np.power(1+market_appreciation_rate, np.arange(duration, 0, -1))*np.power(1+percent_increase_in_amount_over_time, np.arange(0, duration, 1))
    return np.sum(per_year_amount_vect * exponential_factors)

# if PIOT = 1+percent_increase_in_amount_over_time
# The way this works is we basically run:
# and MAR = 1+market_appreciation_rate
# amount*((MAR^10)*(PIOT^0)+(MAR^9)*(PIOT^1)+(MAR^8)*(PIOT^2)...+(MAR^0)*(PIOT^10))
# here the first term is our first year of earnings, which is invested for 10 years, the next term is our 
# next year of earnings which is invested for 9 years, etc. 
    

We also create an income tax calculator so that we can adjust the gross yearly rental income to the net income post taxes.
In the future we should modify this to be based on business taxes not individual income taxes.

In [2]:
def income_tax_calculator(pre_tax_income_amount):
    #TODO: we need to make this more accurate based on taxes in the property zone, I was lazy here
    return pre_tax_income_amount * 0.3

**Macro Parameters:**

In [3]:
MARKET_RATE = 0.05
NUMBER_YEARS_RENTED = 15
APPRECIATION_RATE = 0.06
INITIAL_PROPERTY_COST = 200000
IMMEDIATE_INVESTMENT_COST = 100000
PROPERTY_TAX_RATE = 0.033
RENTAL_RATE_PER_WEEK = 1000
NUMBER_WEEKS_RENTED_PER_YEAR = 50 #todo: remove 4th of july week, and factor in duration in case of winterizing
RENTAL_EFFORT_PER_YEAR = 0 #in human-hours
UPKEEP_COST_PER_YEAR = 2000
TOTAL_INITAL_VALUE_OF_PROPERTY = INITIAL_PROPERTY_COST + IMMEDIATE_INVESTMENT_COST

**Appreciation**

We assume that the property will appreciate at a rate: APPRECIATION_RATE, which we will estimate based on past appreciation rate of the property as well as appreciation rate of other properties in the area.  No need to account for time value of money here since the amount we have when we liquidate the property is the amount we have, there's no question of alternative forms of investment, the time value of the money we invest is locked in at the APPRECIATION_RATE. Here we assume the total initial value of the property is INITIAL_PROPERTY_COST + IMMEDIATE_INVESTMENT_COST, we make the assumption that our initial investment holds its value during future appraisal

In [4]:
appreciated_value_of_property = TOTAL_INITAL_VALUE_OF_PROPERTY*(1+APPRECIATION_RATE)**NUMBER_YEARS_RENTED

**Costs**

These are the costs we will factor in:
    1. Property upkeep
    2. Rental effort (human-hours spent on decisions etc.)
    3. Rental Costs (cleaning, electricity, gas, etc.)
    4. Property Tax (this will be a function of the appreciation rate)


In [5]:
# Property Repairs
# I've read that a rule of thumb is 1% of the appraised value of the property is the repair cost for that year
# we will use our calculator here with an initial amount 1% of the appraised value and have that repair/upkeep increase
#at a rate equal to the APPRECIATION_RATE.  As always, our market_appreciation_rate will be MARKET_RATE.

property_upkeep_total_cost = compute_time_value_of_recurring_cost(amount=0.01*TOTAL_INITAL_VALUE_OF_PROPERTY, 
                                     percent_increase_in_amount_over_time=APPRECIATION_RATE, 
                                     market_appreciation_rate=MARKET_RATE, 
                                     duration=NUMBER_YEARS_RENTED)
costs = property_upkeep_total_cost

In [6]:
# Rental Costs + Rental Effort (Human costs)
# Assumption: no increase in costs over time
TOTAL_RECURRING_RENTAL_COSTS = UPKEEP_COST_PER_YEAR + RENTAL_EFFORT_PER_YEAR
total_value_of_rental_costs = compute_time_value_of_recurring_cost(amount=TOTAL_RECURRING_RENTAL_COSTS, 
                                     percent_increase_in_amount_over_time=0, 
                                     market_appreciation_rate=MARKET_RATE, 
                                     duration=NUMBER_YEARS_RENTED)
costs += total_value_of_rental_costs

In [7]:
# Property Tax
total_value_of_property_taxes = compute_time_value_of_recurring_cost(amount=TOTAL_INITAL_VALUE_OF_PROPERTY*PROPERTY_TAX_RATE, 
                                     percent_increase_in_amount_over_time=APPRECIATION_RATE, 
                                     market_appreciation_rate=MARKET_RATE, 
                                     duration=NUMBER_YEARS_RENTED)
costs += total_value_of_property_taxes

**Income**

These are the income streams we will factor in:
    1. Direct rental income after taxes
    2. Tax discounts

Income will be assumed to appreciate at the MARKET_RATE.

In [8]:
# Direct rental income after taxes
# Assumptions: rental rate per week grows at the same rate as appreciation.  If house appreciates by 2x in 5 years
# the rent will also double.
rental_income = RENTAL_RATE_PER_WEEK * NUMBER_WEEKS_RENTED_PER_YEAR * NUMBER_YEARS_RENTED
rental_income_post_tax = rental_income - income_tax_calculator(rental_income)
income = compute_time_value_of_recurring_cost(amount=rental_income_post_tax, 
                                     percent_increase_in_amount_over_time=APPRECIATION_RATE, 
                                     market_appreciation_rate=MARKET_RATE, 
                                     duration=NUMBER_YEARS_RENTED)

In [9]:
TAX_DISCOUNTS=0
#TODO: ask about these
income += TAX_DISCOUNTS

In [10]:
gross_monetary_end_amount = appreciated_value_of_property + income - costs
time_value_of_expenses = costs + TOTAL_INITAL_VALUE_OF_PROPERTY * (1+MARKET_RATE) ** NUMBER_YEARS_RENTED

total_net_return_on_investment = gross_monetary_end_amount - time_value_of_expenses
ROI = total_net_return_on_investment/(costs + TOTAL_INITAL_VALUE_OF_PROPERTY)


print("property_upkeep_total_cost", property_upkeep_total_cost)
print("total_value_of_rental_costs", total_value_of_rental_costs)
print("total_value_of_property_taxes", total_value_of_property_taxes)
print("rental_income_post_tax", rental_income_post_tax)
print("gross_monetary_end_amount:", gross_monetary_end_amount)
print("appreciated_value_of_property", appreciated_value_of_property)
print("time_value_of_expenses:", time_value_of_expenses)
print("total_net_return_on_investment", total_net_return_on_investment)
print("ROI:", ROI)

property_upkeep_total_cost 100053.45431182167
total_value_of_rental_costs 45314.98353527744
total_value_of_property_taxes 330176.3992290115
rental_income_post_tax 525000.0
gross_monetary_end_amount: 17752777.125422593
appreciated_value_of_property 718967.4579299075
time_value_of_expenses: 1099223.2908995212
total_net_return_on_investment 16653553.834523072
ROI: 21.473360453676417


**Happiness**

More difficult to quantify, happiness is the non-monetary benefit to our family and friends that this property would provide. For example, with this property we would be able to host more family on the 4th of July and other occasions. We would have more waterfront and slightly more privacy from other neighbors. 

A crude estimation of part of this benefit would be the total cost to put the amount of people the real estate property would house into an airbnb nearby, for the expected duration of our ownership. Then we can attempt to monetariy quantify the downside of not staying N miles away vs. staying adjascent to the other properties/main gathering.

**Alternative Investment Calculations**

Below we will review some alternative investment calculations for other real estate options as well as for simple investments in the market at a fixed estimated rate.

In [11]:
#remember to factor in capital gains tax

